In [1]:
import pandas as pd       #importing libraries
import numpy as np

# Data pre-processing

In [2]:
df = pd.read_csv('movie5.csv')   #reading dataset

In [3]:
df

,title,actor,director,genre,desc,poster,details
0,Selfiee,"Akshay Kumar, Nushrratt Bharuccha, Mrunal Thak...",Raj Mehta,"Action, Comedy, Drama",Bollywood superstar Vijay Kumar needs to obtai...,https://m.media-amazon.com/images/M/MV5BYTg5Nm...,https://www.imdb.com/title/tt15516726/?ref_=ad...
1,Pathaan,"Shah Rukh Khan, Deepika Padukone, John Abraham...",Siddharth Anand,"Action, Adventure, Drama",An Indian spy takes on the leader of a group o...,https://m.media-amazon.com/images/M/MV5BM2QzM2...,https://www.imdb.com/title/tt12844910/?ref_=ad...
2,Lost,"Yami Gautam, Piaa Bajpai, Rohit Basfore, Yoges...",Aniruddha Roy Chowdhury,"Drama, Thriller",An emotional thriller that represents a higher...,https://m.media-amazon.com/images/M/MV5BMGU3Yz...,https://www.imdb.com/title/tt15048606/?ref_=ad...
3,3 Idiots,"Aamir Khan, Madhavan, Mona Singh, Kareena Kapoor",Rajkumar Hirani,"Comedy, Drama",Two friends are searching for their long lost ...,https://m.media-amazon.com/images/M/MV5BNTkyOG...,https://www.imdb.com/title/tt1187043/?ref_=adv...
4,Waltair Veerayya,"Chiranjeevi, Ravi Teja, Shruti Haasan, Catheri...",K.S. Ravindra,"Action, Drama","The story between a fisherman and his friend, ...",https://m.media-amazon.com/images/M/MV5BNTE2Zj...,https://www.imdb.com/title/tt15281402/?ref_=ad...
...,...,...,...,...,...,...,...
9445,PK,"Aamir Khan, Anushka Sharma, Sanjay Dutt, Boman...",Rajkumar Hirani,"Comedy, Drama, Sci-Fi",An alien on Earth loses the only device he can...,https://m.media-amazon.com/images/M/MV5BMTYzOT...,https://www.imdb.com/title/tt2338151/?ref_=adv...
9446,Laal Singh Chaddha,"Aamir Khan, Ahmad Ibn Umar, Kareena Kapoor, Ha...",Advait Chandan,"Comedy, Drama, Romance","Laal Singh Chaddha, a simple man whose extraor...",https://m.media-amazon.com/images/M/MV5BMmQ4OT...,https://www.imdb.com/title/tt10028196/?ref_=ad...
9447,Welcome Home,"Kashmira Irani, Swarda Thigale, Shashi Bhushan...",Pushkar Mahabal,"Crime, Drama, Horror",A census report leads two female government te...,https://m.media-amazon.com/images/M/MV5BYjMyZj...,https://www.imdb.com/title/tt13381376/?ref_=ad...
9448,Dangal,"Aamir Khan, Sakshi Tanwar, Fatima Sana Shaikh,...",Nitesh Tiwari,"Action, Biography, Drama",Former wrestler Mahavir Singh Phogat and his t...,https://m.media-amazon.com/images/M/MV5BMTQ4Mz...,https://www.imdb.com/title/tt5074352/?ref_=adv...


In [1]:
df.isnull().sum()       #to calculate null value in each column

In [2]:
df.duplicated().sum()   #to calculate duplicate value in each column

In [5]:
df.drop_duplicates(keep = 'first',inplace = True)      #to drop duplicate value except first one

In [6]:
def convert(x):                          #to convert series into list
    temp = x.replace(',','').split()
    return temp

In [7]:
def converter(x):                       #to convert series into list
    temp = x.split(',')
    return temp

In [47]:
df                 #to view daatset

In [8]:
df['genre'] = df['genre'].apply(convert)                #applying functions on columns
df['actor'] = df['actor'].apply(converter)
df['director'] = df['director'].apply(converter)
df['desc'] = df['desc'].apply(convert)

In [9]:
df['actor'] = df['actor'].apply(lambda x:[i.replace(' ','') for i in x])  #to concatenate first and last word like AkshayKumar
df['director'] = df['director'].apply(lambda x:[i.replace(' ','') for i in x])
df['genre'] = df['genre'].apply(lambda x:[i.replace(' ','') for i in x])

In [10]:
df['tags'] = df['desc'] + df['genre'] + df['actor'] + df['director']   #to make a column tags by combining these columns

In [3]:
df['tags'][15]       #to view tags of movie at specific index(15th index)

In [20]:
new_df = df[['title','tags','poster','genre','details']]       #to make new DataFrame by taking these columns

In [22]:
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x)) #to convert values in a single string

C:\Users\ASUS\AppData\Local\Temp/ipykernel_2364/608913419.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x)) #to convert values in a single string


In [4]:
new_df['tags'][0]

In [24]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower()) #to convert into lowercase

C:\Users\ASUS\AppData\Local\Temp/ipykernel_2364/1206982080.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:x.lower()) #to convert into lowercase


In [32]:
from nltk.stem.porter import PorterStemmer       #For stemming similar words
ps = PorterStemmer()                             #Ex: love for ['loves','love','loved','loving']

def stem(text):
    y = []
    
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [5]:
new_df['tags'][2]      #to view tags of movie at specific index(2nd index)

In [51]:
new_df['tags'] = new_df['tags'].apply(stem)  #applying stem function in tags column

# Text vectorization using Bag of words technique

In [26]:
from sklearn.feature_extraction.text import CountVectorizer        #vectorizing by taking 5000 most frequent occuring words
cv = CountVectorizer(max_features = 5000,stop_words = 'english')

#stopwords means to avoid sentence making words of english like is,and,are etc.

In [27]:
vectors = cv.fit_transform(new_df['tags']).toarray()   #to convert into array

In [6]:
cv.get_feature_names()           #to see the 5000 words that were taken for vectorization

In [41]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vectors)         #to find similarity between each movie

In [7]:
sorted(list(enumerate(similarity[1])),reverse = True,key = lambda x:x[1])[1:6]

#enumerate function provide list of tuples which contain movie_index and their similarity with the selected movie
#sorted is used to sort the movie in descending order by their similarity and after that 5 movie is fetched

In [43]:
def recommend(movie):      #function to recommend movies
    movie_index = new_df[new_df['title'] == movie].index[0] 
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse = True,key = lambda x:x[1])[1:6]
    
    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [60]:
recommend("Golmaal Again") #to call recommend function

Golmaal: Fun Unlimited
Sunday
Golmaal 5
The last drive
Golmaal Returns


In [48]:
import pickle

In [49]:
pickle.dump(new_df.to_dict(),open('movies.pkl','wb'))

In [50]:
pickle.dump(similarity,open('similarity.pkl','wb'))